# Shortest-Path Problem datasets

## 1. Import

In [16]:
import numpy as np
import networkx as nx
import itertools as iter
from scipy.spatial.distance import cdist

from torch_geometric.utils.convert import from_networkx

import pickle

## 2. Utils

In [17]:
def draw(graph, ax):
    """
    Draws the graph as a matplotlib plot.
    Depots are colored in red. Edges that have been
    traveresed
    """

    # draw nodes according to color and position attribute
    pos = nx.get_node_attributes(graph, "coordinates")
    nx.draw_networkx_nodes(
        graph, pos, ax=ax, node_size=100
    )
    labels_pos = {k: (v + np.array([0, 0.03])) for k, v in pos.items()}
    nx.draw_networkx_labels(
        graph, labels_pos, ax=ax
    )

    # draw edges
    nx.draw_networkx_edges(graph, pos)

## 3. Generating Distance Matrices

In [18]:
def build_graph(num_nodes):
    G = nx.empty_graph(num_nodes, create_using=nx.Graph())
    node_position = {
        i: coordinates for i, coordinates in enumerate(np.random.rand(num_nodes, 2))
    }
    nx.set_node_attributes(G, node_position, "coordinates")

    positions = np.array([node_position[index] for index in range(num_nodes)])
    distances = cdist(positions, positions)

    edges = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                edges.append((i,j))
    G.add_edges_from(edges)

    # add distance as edge attribute
    edge_distance_feature = {(u,v):distances[u,v] for u in range(num_nodes) for v in range(num_nodes) if u != v}
    nx.set_edge_attributes(G, edge_distance_feature, 'dist')
    return G, distances

G, distances = build_graph(50)
print(G.number_of_edges())

1225


## 4. Generate dataset

In [25]:
def get_routes_length(routes, dist_mat):
    """
    Returns the length of the routes
    """
    length = 0
    for route in routes:
        for i in range(len(route)-1):
            length += dist_mat[route[i], route[i+1]]
    return length

def build_dataset(size, num_nodes):
    graphs = []
    targets = []
    opts = []
    for i in range(size):
        if i % 10000 == 0:
            print(f'Building graph {i}/{size}')
        graph, dist_mat = build_graph(num_nodes)
        graph_pyg = from_networkx(graph, group_node_attrs=['coordinates'], group_edge_attrs=['dist'])
        sources, destinations = zip(*iter.product(range(num_nodes), repeat=2))
        opt_routes = nx.all_shortest_paths(graph, weight='dist', source=list(sources), target=list(destinations))
        opt_length = get_routes_length(opt_routes, dist_mat)

        # Assign optimum length
        graph_pyg.y = opt_length

        # Saving
        graphs.append(graph_pyg)
        targets.append(opt_routes)
        opts.append(opt_length)
    return graphs, targets, opts

In [26]:
data_20_train = build_dataset(50000,20)
with open('PATH_dataset_20_train.pkl', 'wb') as file:
    pickle.dump(data_20_train, file)

Building graph 0/50000


NodeNotFound: Node [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 

In [ ]:
data_20_test = build_dataset(100,20)
with open('PATH_dataset_20_test.pkl', 'wb') as file:
    pickle.dump(data_20_test, file)

In [ ]:
data_50_train = build_dataset(70000,50)
with open('PATH_dataset_50_train.pkl', 'wb') as file:
    pickle.dump(data_50_train, file)

In [ ]:
data_50_test = build_dataset(100,50)
with open('PATH_dataset_50_test.pkl', 'wb') as file:
    pickle.dump(data_50_test, file)